In [1]:
from pyTsetlinMachineParallel.tm import MultiClassTsetlinMachine
from tmu.tsetlin_machine import TMCoalescedClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np 
import pickle
import torch
import tqdm
from collections import Counter, defaultdict
!export OMP_NUM_THREADS=10
n_clauses_per_class = 200
n_clauses_per_class = 200

treshold = int(n_clauses_per_class * 0.8)
s_param = 6.8
s_param = 21.3
s_param = 11.40000000000001
"s henger sammen hvor ofte du ser pattern"
"lav s sier at man ser et pattern ofte, og kan glemme dette"
"belonning er s/(s-1)"
"random penalty er 1/s"

'random penalty er 1/s'

In [2]:
# tm = MultiClassTsetlinMachine(n_clauses_per_class, treshold, s_param, weighted_clauses=True, boost_true_positive_feedback=0)
tm = TMCoalescedClassifier(n_clauses_per_class, treshold, s_param, weighted_clauses=True, boost_true_positive_feedback=0)
cuda_tm = TMCoalescedClassifier(n_clauses_per_class, treshold, s_param,  platform='CUDA', boost_true_positive_feedback=0)


In [10]:
rf = RandomForestClassifier()

In [3]:
game_amount = 100000
with open("/home/jaoi/master22/pet_for_sale/winning_games_db/{}_tsetlined_games.pkl".format(game_amount), "rb") as fp:
        games = pickle.load(fp)
print(games[:3])
print(games[0][0])
games[0].shape

[[array([1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1.,
         1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0.,
         1., 0., 0., 0., 0., 0., 0.], dtype=float32)
  0]
 [array([1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1.,
         0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
         1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 1

(2,)

In [5]:
games[0][0].shape

(126,)

In [4]:
train_i = int(games.shape[0] * 0.9)
train_i

514835

/bin/bash: nvcc: command not found


In [143]:
ys = Counter()
for game in games:
    ys[game[1]] += 1
ys

Counter({0: 200687,
         4: 28227,
         10: 48887,
         9: 73117,
         12: 46446,
         11: 21266,
         13: 5810,
         1: 61140,
         3: 29043,
         6: 40378,
         5: 12874,
         7: 1322,
         2: 767,
         8: 918,
         15: 282,
         14: 396,
         16: 350,
         17: 91,
         18: 38})

In [5]:
x_train =np.array([game[0] for game in games[:train_i, :1]])
x_test =np.array([game[0] for game in games[train_i:, :1]])

y_train =np.array([game[0] for game in games[:train_i, 1:]])
y_test =np.array([game[0] for game in games[train_i:, 1:]])


# x_test = games[train_i:, :1]

# y_train = games[:train_i, 1:]
# y_test = games[train_i:, 1:]

# for game in games:
#     print(game[0])
#     break

In [10]:
y_train.shape

(514835,)

/bin/bash: nvcc: command not found


In [6]:
epochs = 1
# for epoch in range(epochs):
#     tm.fit(x_train, y_train)

# print("Accuracy tm:", 100*(tm.predict(x_test) == y_test).mean())

for epoch in range(epochs):
    cuda_tm.fit(x_train, y_train)

print("Accuracy tm:", 100*(cuda_tm.predict(x_test) == y_test).mean())

Accuracy tm: 94.04412278861618


In [42]:
print(cuda_tm.weight_banks[0].get_weights().shape[0])
saving_weights = []
for classe in range(cuda_tm.number_of_classes):
    saving_weights.append(cuda_tm.weight_banks[classe].get_weights())

200


In [20]:
tsetlin_name = "1r3r_200_clauses_3_epochs"
with open("tsetlin_models/{}.pickle".format(tsetlin_name), "wb") as fp:
    pickle.dump(saving_weights, fp)

In [21]:
rf.fit(x_train, y_train)
print("Accuracy rf:", 100*(rf.predict(x_test) == y_test).mean())

NameError: name 'rf' is not defined

In [13]:
def tsetlin_hyperparameter_search(n_clauses_per_class, x_t, y_t, x_v, y_v):
    """
    Search for the best hyperparameters for the tsetlin machine
    """
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best = 0.0
    treshold = int(0.8 * n_clauses_per_class)
    print("starting training")
    for s_param in tqdm.tqdm(np.arange(1.0, 30.0, 0.1)):
        tm = MultiClassTsetlinMachine(n_clauses_per_class, treshold, s_param, weighted_clauses=True, boost_true_positive_feedback=0)
        tm.fit(x_t, y_t, epochs=200)
        acc = 100*(tm.predict(x_v) == y_v).mean()
        if acc > best:
            print("NEW BEST ACC {} WITH s_param {}".format(acc, s_param))
            best = acc
# tsetlin_hyperparameter_search(50, x_train, y_train, x_test, y_test)

In [22]:
cuda_tm.number_of_classes

19

In [7]:
def get_name_of_feature_from_3_rounds(feature_nr):
    # for own player with 4 bits representing the cards as max number is 11
    reverse_4_bits = [3, 2, 1, 0]
    reverse_5_bits = [4, 3, 2, 1, 0]
    reverse_2_bits = [1, 0]
    if feature_nr in [0,1,2,3,4]:
        return "own coins bit {}".format(2**(reverse_5_bits[feature_nr]))
    elif feature_nr in [5,6,7,8]:
        return "own cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 5])
    elif feature_nr in [9,10,11,12]:
        return "own cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 9])
    elif feature_nr in [13,14,15,16]:
        return "own cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 13])
    elif feature_nr == 17:
        return "self can bid bit"
    elif feature_nr in [18,19,20,21,22]:
        return "current own bid {}".format(2**reverse_5_bits[feature_nr - 18])
    elif feature_nr in [23,24,25,26,27]:
        return "resulting own coins if fold {}".format(2**reverse_5_bits[feature_nr - 23])
    elif feature_nr in [28,29,30,31]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 28])
    elif feature_nr in [32,33]:
        return "next turn order {}".format(2**reverse_2_bits[feature_nr - 32])
    # for next player in line
    elif feature_nr in [34, 35, 36, 37, 38]:
        return "next player coins bit {}".format(2**(reverse_5_bits[feature_nr - 34]))
    elif feature_nr in [39, 40, 41, 42]:
        return "next player cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 39])
    elif feature_nr in [43, 44, 45, 46]:
        return "next player cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 43])
    elif feature_nr in [47, 48, 49, 50]:
        return "next player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 47])
    elif feature_nr == 51:
        return "next can bid bit"
    elif feature_nr in [52, 53, 54, 55, 56]:
        return "current next player bid {} bit".format(2**reverse_5_bits[feature_nr - 52])
    elif feature_nr in [57, 58, 59, 60, 61]:
        return "resulting next player coins if fold {}".format(2**reverse_5_bits[feature_nr - 57])
    elif feature_nr in [62, 63, 64, 65]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 62])
    elif feature_nr in [66, 67]:
        return "next turn order {}".format(2**reverse_2_bits[feature_nr - 66])
    # for last player in line
    elif feature_nr in [68, 69, 70, 71, 72]:
        return "last player coins bit {}".format(2**(reverse_5_bits[feature_nr - 68]))
    elif feature_nr in [73, 74, 75, 76]:
        return "last player cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 73])
    elif feature_nr in [77, 78, 79, 80]:
        return "last player cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 77])
    elif feature_nr in [81, 82, 83, 84]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 81])
    elif feature_nr == 85:
        return "last player can bid"
    elif feature_nr in [86, 87, 88, 89, 90]:
        return "current next player bid {}".format(2**reverse_5_bits[feature_nr - 86])
    elif feature_nr in [91, 92, 93, 94, 95]:
        return "resulting next player coins if fold {}".format(2**reverse_5_bits[feature_nr - 91])
    elif feature_nr in [96, 97, 98, 99]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 96])
    elif feature_nr in [100, 101]:
        return "next turn order {}".format(2**reverse_2_bits[feature_nr - 100])
    # remainder stats
    elif feature_nr in [102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112]:
        return "{}. largest in stack".format(feature_nr - 101)
    elif feature_nr in [113, 114, 115, 116]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 113])
    elif feature_nr in [117, 118, 119, 120]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 117])
    elif feature_nr in [121, 122, 123, 124]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 121])
    elif feature_nr == 125:
        return "board_std"
    return "not implemented"

In [16]:
sum([5, 4, 4, 4, 1, 5, 5, 4, 2]) * 3 + 11 + 4 + 4 + 4 + 1

126

In [17]:
sum([5, 4, 4, 4, 1, 5, 5, 4, 2])

34

In [8]:
cuda_tm.weight_banks[0].weights.shape

(200,)

In [9]:
def show_clauses_from_tm_standard_3_rounds(tm, one_class=False):
    # from pytsetlin:
    # number_of_features = int(tm.number_of_features/2)
    number_of_features = 126
    number_of_classes = tm.number_of_classes
    if not one_class:
        for clas in range(number_of_classes):

            print("\nClass {} Positive Clauses:\n".format(clas))
            for j in range(0, n_clauses_per_class, 2):
                print("Clause #%d: " % (j), end=' ')
                l = []
                for k in range(number_of_features*2):
                    if tm.get_ta_action(clas, j, k) == 1:
                        if k < number_of_features:
                            l.append(" "+get_name_of_feature_from_3_rounds(k))
                        else:
                            l.append("¬"+get_name_of_feature_from_3_rounds(k-number_of_features))
                print(" ∧ ".join(l))

            print("\nClass {} Negative Clauses:\n".format(clas))
            for j in range(1, n_clauses_per_class, 2):
                print("Clause #%d: " % (j), end=' ')
                l = []
                for k in range(number_of_features*2):
                    if tm.get_ta_action(clas, j, k) == 1:
                        if k < number_of_features:
                            l.append(" "+get_name_of_feature_from_3_rounds(k))
                        else:
                            l.append("¬"+get_name_of_feature_from_3_rounds(k-number_of_features))
                print(" ∧ ".join(l))
    else:
        

        print("\nClass {} Positive Clauses:\n".format(one_class))
        for j in range(0, n_clauses_per_class, 2):
            print("Clause #%d: " % (j), end=' ')
            l = []
            for k in range(number_of_features*2):
                if tm.get_ta_action(one_class, j, k) == 1:
                    if k < number_of_features:
                        l.append(" "+get_name_of_feature_from_3_rounds(k))
                    else:
                        l.append("¬"+get_name_of_feature_from_3_rounds(k-number_of_features))
            print(" ∧ ".join(l))

        print("\nClass {} Negative Clauses:\n".format(one_class))
        for j in range(1, n_clauses_per_class, 2):
            print("Clause #%d: " % (j), end=' ')
            l = []
            for k in range(number_of_features*2):
                if tm.get_ta_action(one_class, j, k) == 1:
                    if k < number_of_features:
                        l.append(" "+get_name_of_feature_from_3_rounds(k))
                    else:
                        l.append("¬"+get_name_of_feature_from_3_rounds(k-number_of_features))
            print(" ∧ ".join(l))

show_clauses_from_tm_standard_3_rounds(cuda_tm, one_class=12)


Class 12 Positive Clauses:

Clause #0:  

TypeError: get_ta_action() takes 3 positional arguments but 4 were given

In [10]:
def make_clauses_into_rule_stats(tm):
    number_of_features = int(tm.number_of_features/2)
    number_of_classes = tm.number_of_classes
    
    analyzed_classes = {}
    for clas in range(number_of_classes):
        pos = Counter()
        neg = Counter()
        
        # print("\nClass {} Positive Clauses:\n".format(clas))
        for j in range(0, n_clauses_per_class, 2):
            # print("Clause #%d: " % (j), end=' ')
            for k in range(number_of_features*2):
                if tm.ta_action(clas, j, k) == 1:
                    if k < number_of_features:
                        pos[get_name_of_feature_from_3_rounds(k)] += 1
                    else:
                        pos["¬"+get_name_of_feature_from_3_rounds(k-number_of_features)] += 1

        # print("\nClass {} Negative Clauses:\n".format(clas))
        for j in range(1, n_clauses_per_class, 2):
            # print("Clause #%d: " % (j), end=' ')
            l = []
            for k in range(number_of_features*2):
                if tm.ta_action(clas, j, k) == 1:
                    if k < number_of_features:
                        neg[get_name_of_feature_from_3_rounds(k)] += 1
                    else:
                        neg["¬"+get_name_of_feature_from_3_rounds(k-number_of_features)] += 1
        
        analyzed_classes[clas] = {"negative": neg, "positive": pos}
    return analyzed_classes

analyzed = make_clauses_into_rule_stats(tm)


AttributeError: 'TMCoalescedClassifier' object has no attribute 'number_of_features'

In [10]:
# prev1 = 0
# prev2 = 0
# for k, v in analyzed.items():
#     print("*"*5 + f"{k}" + "*"*5)
#     print("positive")
#     print(v["positive"].most_common(10))
#     print("negative")
#     print(v["negative"].most_common(10))  
    

In [67]:
# gives out a list of vectores of length number of features where each vector is number_of_classes * n_clauses_per_class long
# tm.transform(x_test[0]).shape

# cuda_tm.transform(x_test[0]).shape
cuda_tm.transform(x_test[0])[0]
# cuda_tm.predict(x_test[0].reshape(1, -1))

# len(cuda_tm.weight_banks)

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0], dtype=uint32)

In [11]:
from collections import Counter



def new_bent_predict(cuda_tm, x, ret_dic=False):
    total_vote = Counter()
    for j, classe in enumerate(range(cuda_tm.number_of_classes)):
        class_vote = []
        
        for i, clauses in enumerate(cuda_tm.transform(x)):
            # transform transformes all the features into a list of clauses with len == num_features

            for clause in clauses:
                # each clause is multiplied with the weight and added to the list of class_votes 
                class_vote.append(cuda_tm.get_weight(classe, clause) * clause)
            # the votes for each class are then counted and returned
        total_vote[classe] += sum(class_vote)
    if ret_dic:
        return total_vote
    else:
        return total_vote.most_common()[0][0]

for i, x_arr in enumerate(x_test):
    og_predict = cuda_tm.predict(x_arr.reshape(1, -1))[0]
    bent_pred = new_bent_predict(cuda_tm, x_arr)
    if og_predict != bent_pred:
        print("SHIT ITS WRONG ON {}".format(i))
        print("OG SAYS {}, BENT SAYS {}".format(og_predict, bent_pred))
        print(new_bent_predict(cuda_tm, x_arr, True))



SHIT ITS WRONG ON 0
OG SAYS 0, BENT SAYS 12
Counter({12: 267120, 11: 167031, 16: -10080, 18: -15120, 7: -22437, 17: -24930, 14: -25200, 6: -27720, 8: -30240, 2: -35280, 9: -39872, 15: -39872, 3: -42364, 5: -59832, 4: -73080, 1: -74790, 13: -104706, 10: -133560, 0: -176400})
SHIT ITS WRONG ON 1
OG SAYS 10, BENT SAYS 12
Counter({12: 266060, 11: 167701, 16: -10040, 18: -15060, 7: -22527, 17: -25030, 14: -25100, 6: -27610, 8: -30120, 2: -35140, 9: -40048, 15: -40048, 3: -42551, 5: -60072, 4: -72790, 1: -75090, 13: -105126, 10: -133030, 0: -175700})
SHIT ITS WRONG ON 2
OG SAYS 0, BENT SAYS 12
Counter({12: 263092, 11: 168572, 16: -9928, 18: -14892, 7: -22644, 14: -24820, 17: -25160, 6: -27302, 8: -29784, 2: -34748, 9: -40256, 15: -40256, 3: -42772, 5: -60384, 4: -71978, 1: -75480, 13: -105672, 10: -131546, 0: -173810})
SHIT ITS WRONG ON 3
OG SAYS 4, BENT SAYS 12
Counter({12: 266166, 11: 166361, 16: -10044, 18: -15066, 7: -22347, 17: -24830, 14: -25110, 6: -27621, 8: -30132, 2: -35154, 9: -39

KeyboardInterrupt: 

In [7]:
def show_coalesced_clause(tm, clause):
    number_of_features = 126
    print("Clause #%d: " % (clause), end=' ')
    l = []
    for k in range(number_of_features*2):
        if tm.get_ta_action(clause, k) == 1:
            if k < number_of_features:
                l.append(" "+get_name_of_feature_from_3_rounds(k))
            else:
                l.append("¬"+get_name_of_feature_from_3_rounds(k-number_of_features))
    print(" ∧ ".join(l))

def show_coalesced_weight(tm, the_class, weight):
    mw = tm.weight_banks[the_class].get_weights()
    print(mw[weight])
    
    
    

In [8]:
def show_coalesced_clauses(tm):
    number_of_features = 126
    n_clauses = tm.weight_banks[0].get_weights().shape[0]
    for j in range(n_clauses):
            print("Clause #%d: " % (j), end=' ')
            l = []
            for k in range(number_of_features*2):
                if tm.get_ta_action(j, k) == 1:
                    if k < number_of_features:
                        l.append(" "+get_name_of_feature_from_3_rounds(k))
                    else:
                        l.append("¬"+get_name_of_feature_from_3_rounds(k-number_of_features))
            print(" ∧ ".join(l))
            
# show_coalesced_clauses(cuda_tm)

In [9]:
def bin_numpy_to_number(bin_array):
    bin_str = ""
    if type(bin_array) == np.float32:
        bin_str += str(int(bin_array))        
    else:
        for digit in bin_array:
            bin_str += str(int(digit))
    dec_num = int(bin_str, base=2)
    return dec_num

def reverse_onehot(target, stack_len):
    stack = np.zeros(stack_len)
    actives = np.argwhere(target == 1)
    for i, act in enumerate(actives):
        stack[i] = stack_len - act
    return stack

def convert_3_rounds_fs1_bin_obs_to_obs(obs):
    stack_len = 11
    new_obs = np.zeros(42)
    new_obs[0] = bin_numpy_to_number(obs[0:5])
    new_obs[1] = bin_numpy_to_number(obs[5:9])
    new_obs[2] = bin_numpy_to_number(obs[9:13])
    new_obs[3] = bin_numpy_to_number(obs[13:17])
    new_obs[4] = bin_numpy_to_number(obs[17])
    new_obs[5] = bin_numpy_to_number(obs[18:23])
    new_obs[6] = bin_numpy_to_number(obs[23:28])
    new_obs[7] = bin_numpy_to_number(obs[28:32])
    new_obs[8] = bin_numpy_to_number(obs[32:34])
    new_obs[9] = bin_numpy_to_number(obs[34:39])
    new_obs[10] = bin_numpy_to_number(obs[39:43])
    new_obs[11] = bin_numpy_to_number(obs[43:47])
    new_obs[12] = bin_numpy_to_number(obs[47:51])
    new_obs[13] = bin_numpy_to_number(obs[51])
    new_obs[14] = bin_numpy_to_number(obs[52:57])
    new_obs[15] = bin_numpy_to_number(obs[57:62])
    new_obs[16] = bin_numpy_to_number(obs[62:66])
    new_obs[17] = bin_numpy_to_number(obs[66:68])
    new_obs[18] = bin_numpy_to_number(obs[68:73])
    new_obs[19] = bin_numpy_to_number(obs[73:77])
    new_obs[20] = bin_numpy_to_number(obs[77:81])
    new_obs[21] = bin_numpy_to_number(obs[81:85])
    new_obs[22] = bin_numpy_to_number(obs[85])
    new_obs[23] = bin_numpy_to_number(obs[86:91])
    new_obs[24] = bin_numpy_to_number(obs[91:96])
    new_obs[25] = bin_numpy_to_number(obs[96:100])
    new_obs[26] = bin_numpy_to_number(obs[100:102])
    new_obs[27:38] = reverse_onehot(obs[102:113], stack_len)
    new_obs[38] = bin_numpy_to_number(obs[113:117])
    new_obs[39] = bin_numpy_to_number(obs[117:121])
    new_obs[40] = bin_numpy_to_number(obs[121:125])
    new_obs[41] = np.std(new_obs[38:41])
    
    return np.array(new_obs, dtype="float32")

def make_observation_into_gamestate(obs):
    # own is first, then next player, then final player
    new_obs = obs.astype(int)
    print("Current board {}".format(new_obs[38:41]))    
    print("Player to move coins: {}".format(new_obs[0]))
    print("Player to move cards: {}".format(new_obs[1:4]))
    print("Player to move can bid: {}".format(bool(new_obs[4])))
    print("Player to move current bid: {}".format(new_obs[5]))
    print("Player to move coins on fold: {}".format(new_obs[6]))
    print("Player to move achieved card on fold: {}".format(new_obs[7]))
    print("Player to move moving order if all folds now: {}".format(new_obs[8]))
    print("Next player coins: {}".format(new_obs[9]))
    print("Next player cards: {}".format(new_obs[10:13]))
    print("Next player can bid: {}".format(bool(new_obs[13])))
    print("Next player current bid: {}".format(new_obs[14]))
    print("Next player coins on fold: {}".format(new_obs[15]))
    print("Next player achieved card on fold: {}".format(new_obs[16]))
    print("Next player moving order if all folds now: {}".format(new_obs[17]))
    print("Final player coins: {}".format(new_obs[18]))
    print("Final player cards: {}".format(new_obs[19:22]))
    print("Final player can bid: {}".format(bool(new_obs[22])))
    print("Final player current bid: {}".format(new_obs[23]))
    print("Final player coins on fold: {}".format(new_obs[24]))
    print("Final player card on fold: {}".format(new_obs[25]))
    print("Final player moving order if all folds now: {}".format(new_obs[26]))
    print("Current stack: {}".format(new_obs[27:38]))


In [10]:
the_class = 0
mw = cuda_tm.weight_banks[the_class].get_weights()
mw.shape
c_out = cuda_tm.transform(x_test[0].reshape(1, -1))
# print(c_out)
clauses_out = np.where(c_out == 1)[1]
for clas in range(len(cuda_tm.weight_banks)):
    # print("for class {}".format(clas))
    for clause in clauses_out:
        pass
        # show_coalesced_clause(cuda_tm, clause)
        # show_coalesced_weight(cuda_tm, clas, clause)
        
    #TODO find the weight for each clause as well
c_out.shape
np.dot(c_out, mw)

def compare_predictions(tm, x):
    c_out = cuda_tm.transform(x.reshape(1, -1))
    moves_and_scores = []
    classes = len(tm.weight_banks)
    for clas in range(classes):
        mw = cuda_tm.weight_banks[clas].get_weights()
        # print(np.argpartition(mw, -5)[-5:])
        moves_and_scores.append((clas, np.dot(c_out, mw)))
    top_moves = sorted(moves_and_scores, reverse=True, key=lambda x:x[1])

    if top_moves[0][0] != tm.predict(x.reshape(1, -1))[0]:
        print("OH OH <STINKY!> {} {}".format(top_moves[:5], tm.predict(x.reshape(1, -1))[0]))
        return True
    else:
        False
    
wrongs = 0
for i in range(10000):
    if compare_predictions(cuda_tm, x_test[i]):
        print("at i: {}".format(i))
        wrongs += 1
print("wrongs!!!! {}".format(wrongs))
    
def get_top_5_actions_for_state(tm, x):
    moves_and_scores = []
    c_out = cuda_tm.transform(x.reshape(1, -1))
    classes = len(tm.weight_banks)
    for clas in range(classes):
        mw = cuda_tm.weight_banks[clas].get_weights()
        # print(np.argpartition(mw, -5)[-5:])
        moves_and_scores.append((clas, np.dot(c_out, mw)))
    
    top_moves = sorted(moves_and_scores, reverse=True, key=lambda x:x[1])
    votes = []
    for move, vote in top_moves[0:5]:
        votes.append(vote)
    return top_moves[0:5], np.std(votes), c_out 


for i in range(10000):
    result, std = get_top_5_actions_for_state(cuda_tm, x_test[i])
    # if result[0][1] - result[1][1] < 10:
    #     print("LOW CERTAINTY DECISION AT {}".format(i), result)
    #     make_observation_into_gamestate(convert_3_rounds_fs1_bin_obs_to_obs(x_test[i]))
    #     print()
    if result[0][1] > treshold and result[1][1] > treshold:
        print("VOTE DIFFICULTY AT {}".format(i), result)
        make_observation_into_gamestate(convert_3_rounds_fs1_bin_obs_to_obs(x_test[i]))
        print()
    # if std < 30:
    #     print("STD LOWER THAN 30 ", result)


OH OH <STINKY!> [(6, array([267])), (5, array([192])), (0, array([-99])), (7, array([-225])), (8, array([-237]))] 5
at i: 195
OH OH <STINKY!> [(6, array([240])), (5, array([181])), (0, array([-92])), (7, array([-231])), (8, array([-235]))] 5
at i: 775
OH OH <STINKY!> [(5, array([181])), (1, array([170])), (3, array([43])), (9, array([-35])), (0, array([-49]))] 1
at i: 1226
OH OH <STINKY!> [(6, array([189])), (5, array([173])), (9, array([81])), (8, array([-229])), (7, array([-252]))] 5
at i: 1667
OH OH <STINKY!> [(6, array([217])), (5, array([205])), (0, array([-97])), (7, array([-190])), (8, array([-232]))] 5
at i: 2706
OH OH <STINKY!> [(9, array([196])), (6, array([169])), (5, array([103])), (0, array([-47])), (7, array([-125]))] 6
at i: 3111
OH OH <STINKY!> [(11, array([232])), (10, array([202])), (0, array([12])), (3, array([-19])), (12, array([-80]))] 10
at i: 3214
OH OH <STINKY!> [(6, array([220])), (5, array([179])), (0, array([-90])), (9, array([-144])), (4, array([-174]))] 5
a

In [15]:
print(x_train.shape)

[1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1.
 0. 0. 0. 0. 1. 1.]


In [22]:
tm.number_of_classes * n_clauses_per_class


3800

In [74]:
# needs to be 2d array in transform/predict
xn = x_test[0].reshape(1, -1)
cn = tm.transform(xn)
# tm.predict(x_test[0].reshape(1, -1))
# xn.shape
weight0 = tm.get_state()[0][0]
cn[0, 0:200].sum()
weight0 * cn[0,0:200]
weight0

array([ 12,  13,  31,   5,  30,   4,   6,   4,  13,   1,  63,   5,  23,
         5,  19,   5,  12,  12,  12,  17,  11,  12,  26,  24,  20,  14,
         8,  20,  10,  42,  16,  16,  15,   4,  16,  27,  61,  18,  29,
         1,   4,   4,  18,  13,  24,  22,   5,  12,  16,  21,  13,  16,
         6,   7,  18,  15,  11,  33,  15,   6,  26,  29,  27,   1,  51,
        13,   5,  29,  35,  24,  20,  19,  14,  14,  29,  36,  11,   7,
        38,   4,  51,  34,   5,  28,  15,  27,  19,   3,  30,  94,   6,
        36,  18,  56,  33,   8,  23,  23,  17,   6,   8,   1,  16,  28,
        39,   9,  27,  10,  14,  17,   7, 100,  15,   4,  22,  12,  15,
         1,  16,  15,  51,  22,   5,   4,   6,  12,   3,  39,  23,   2,
        10,  33,   9,   3,  10,  16,   7,   3,   5,   4,  10,   4,  16,
        19,  12,  23,  27,   8,  17,  24,  21,   6,  26,   2,  27,  45,
        36,  18,   3,  17,   6,   1,  19,   1,  20,  25,   7,   1,  34,
        17,  25,   1,   6,  85,  18,   7,   5,  15,  16,  18,   

In [109]:
tm.get_state()[0][0]
# cn

array([ 12,  13,  31,   5,  30,   4,   6,   4,  13,   1,  63,   5,  23,
         5,  19,   5,  12,  12,  12,  17,  11,  12,  26,  24,  20,  14,
         8,  20,  10,  42,  16,  16,  15,   4,  16,  27,  61,  18,  29,
         1,   4,   4,  18,  13,  24,  22,   5,  12,  16,  21,  13,  16,
         6,   7,  18,  15,  11,  33,  15,   6,  26,  29,  27,   1,  51,
        13,   5,  29,  35,  24,  20,  19,  14,  14,  29,  36,  11,   7,
        38,   4,  51,  34,   5,  28,  15,  27,  19,   3,  30,  94,   6,
        36,  18,  56,  33,   8,  23,  23,  17,   6,   8,   1,  16,  28,
        39,   9,  27,  10,  14,  17,   7, 100,  15,   4,  22,  12,  15,
         1,  16,  15,  51,  22,   5,   4,   6,  12,   3,  39,  23,   2,
        10,  33,   9,   3,  10,  16,   7,   3,   5,   4,  10,   4,  16,
        19,  12,  23,  27,   8,  17,  24,  21,   6,  26,   2,  27,  45,
        36,  18,   3,  17,   6,   1,  19,   1,  20,  25,   7,   1,  34,
        17,  25,   1,   6,  85,  18,   7,   5,  15,  16,  18,   

In [126]:
def bent_predict(x, tm):
    """
    tsetlin machine works by majority vote, each clause votes either positive for a class or a negative for a class
    The class with the highest vote clause vote is chosen.
    """
    
    n_clauses_per_class = tm.number_of_clauses
    classes = tm.number_of_classes
    # print(classes)
    # print(tm.get_state()[classes+1][0])
    signs = np.ones(n_clauses_per_class * classes)
    a = range(1, signs.shape[0], 2)
    signs[a] = -1
    print(signs.shape)
    cn = tm.transform(x)
    sig_cn = (cn * signs)[0]
    print(sig_cn.shape)
    # sig_cn.tolist()
    
    # weight*signs*cn
    class_vote = []
    for clas in range(classes):
        weight = tm.get_state()[clas][0]
        print(weight.shape)
        starting_clause = clas * n_clauses_per_class
        ending_clause = starting_clause + n_clauses_per_class
        weighted_sig_cn = sig_cn[starting_clause: ending_clause]*weight
        # weighted_sig_cn = sig_cn[starting_clause: ending_clause]
        
        
        class_vote.append((weighted_sig_cn.sum()))
        # print(clas * n_clauses_per_class, clas * n_clauses_per_class + n_clauses_per_class)
    return class_vote

print(bent_predict(xn, tm))

print(xn.shape)
tm.predict(xn)
# cn.shape

(3800,)
(3800,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
[-179.0, 836.0, -913.0, 18.0, -67.0, 381.0, -239.0, -695.0, -834.0, 199.0, -417.0, -392.0, -616.0, -669.0, -1190.0, -1156.0, -892.0, -1083.0, -695.0]
(1, 126)


array([0], dtype=uint32)